# 第4章: データと指示の分離

- [レッスン](#lesson)
- [演習](#exercises)
- [例のプレイグラウンド](#example-playground)

## セットアップ

次のセットアップセルを実行して、APIキーをロードし、`get_completion`ヘルパー関数を確立します。

In [ ]:
# Pythonの組み込み正規表現ライブラリをインポート
import re
import boto3
from botocore.exceptions import ClientError
import json

# utilsパッケージからhintsモジュールをインポート
from utils import hints

# IPythonストアからMODEL_NAME変数を取得
%store -r modelId
%store -r region

bedrock_client = boto3.client(service_name='bedrock-runtime', region_name=region)

In [ ]:
def get_completion(prompt, system_prompt=None):
    # 推論設定を定義します
    inference_config = {
        "temperature": 0.0,
        "maxTokens": 200
    }
    # APIパラメータを設定します
    converse_api_params = {
        "modelId": modelId,
        "messages": [{"role": "user", "content": [{"text": prompt}]}],
        "inferenceConfig": inference_config
    }
    if system_prompt:
        # システムプロンプトがある場合は追加します
        converse_api_params["system"] = [{"text": system_prompt}]
    try:
        # APIにリクエストを送信します
        response = bedrock_client.converse(**converse_api_params)
        text_content = response['output']['message']['content'][0]['text']
        return text_content

    except ClientError as err:
        # エラーメッセージを出力します
        message = err.response['Error']['Message']
        print(f"クライアントエラーが発生しました: {message}")

---

## レッスン

しばしば、完全なプロンプトを書くのではなく、**後で追加の入力データで修正できるプロンプトテンプレートが必要です**。これは、Claudeに毎回同じことをさせたいが、Claudeがそのタスクに使用するデータが毎回異なる場合に便利です。

幸いなことに、**プロンプトの固定された骨組みを変数のユーザー入力から分離し、ユーザー入力をプロンプトに置き換えることで、これを非常に簡単に行うことができます**。 

以下では、置き換え可能なプロンプトテンプレートの書き方と、ユーザー入力をどのように置き換えるかをステップバイステップで説明します。

### 例

この最初の例では、Claudeに動物の鳴き声を生成するように依頼しています。Claudeに送信された完全なプロンプトは、入力（この場合は「Cow」）で置き換えられた`PROMPT_TEMPLATE`だけであることに注意してください。「Cow」という単語は、完全なプロンプトを印刷する際にf-stringを介して`ANIMAL`プレースホルダーを置き換えます。

**注意:** 実際には、プレースホルダー変数に特に何かを呼ぶ必要はありません。この例では`ANIMAL`と呼びましたが、`CREATURE`や`A`と呼ぶことも簡単にできます（ただし、一般的には変数名を特定のものにしておくことが良いです。そうすることで、置き換えなしでもプロンプトテンプレートが理解しやすくなり、ユーザーが解析しやすくなります）。変数に付けた名前が、プロンプトテンプレートのf-stringで使用するものであることを確認してください。

In [ ]:
# 変数の内容
ANIMAL = "Cow"

# 変数の内容のためのプレースホルダーを持つプロンプトテンプレート
PROMPT = f"I will tell you the name of an animal. Please respond with the noise that animal makes. {ANIMAL}"

# Claudeの応答を出力
print("--------------------------- 変数置換を含む完全なプロンプト ---------------------------")
print(PROMPT)
print("\n------------------------------------- Claudeの応答 -------------------------------------")
print(get_completion(PROMPT))

なぜこのように入力を分離して置き換えたいのでしょうか？それは、**プロンプトテンプレートが繰り返しの作業を簡素化するからです**。例えば、第三者のユーザーがプロンプトにコンテンツを提出するように招待するプロンプト構造を構築したとしましょう（この場合、生成したい音の動物です）。これらの第三者ユーザーは、完全なプロンプトを書く必要も、見る必要もありません。彼らがしなければならないのは、変数を埋めることだけです。

ここでは、変数とf-stringsを使用してこの置き換えを行いますが、`format()`メソッドを使ってもできます。

**注意:** プロンプトテンプレートには、望むだけの変数を含めることができます！

このように代入変数を導入する際には、**Claudeが変数の開始と終了を理解していることを確認することが非常に重要です**（指示やタスクの説明と対比して）。以下の例を見てみましょう。ここでは、指示と代入変数の間に区切りがありません。

私たち人間の目には、以下のプロンプトテンプレートで変数がどこで始まり、どこで終わるかは非常に明確です。しかし、完全に代入されたプロンプトでは、その区切りが不明瞭になります。

In [ ]:
# 変数の内容
EMAIL = "Show up at 6am tomorrow because I'm the CEO and I say so."

# 変数の内容のためのプレースホルダーを持つプロンプトテンプレート
PROMPT = f"Yo Claude. {EMAIL} <----- このメールをより丁寧にしてくださいが、それ以外は何も変更しないでください。"

# Claudeの応答を出力
print("--------------------------- 変数置換を含む完全なプロンプト ---------------------------")
print(PROMPT)
print("\n------------------------------------- Claudeの応答 -------------------------------------")
print(get_completion(PROMPT))

ここで、**Claudeは「Yo Claude」が書き直すべきメールの一部だと考えています**！それは、書き直しが「Dear Claude」で始まることからわかります。人間の目には明らかですが、特にメールが始まりと終わりを示すプロンプトテンプレートでは明確ですが、置換後のプロンプトでははるかに不明瞭になります。

この問題をどう解決しますか？**入力をXMLタグで囲みます**！以下のように行いましたが、出力にはもう「Dear Claude」が含まれていないことがわかります。

[XMLタグ](https://docs.anthropic.com/claude/docs/use-xml-tags)は、`<tag></tag>`のような角括弧のタグです。これらは対になっており、開くタグ（例：`<tag>`）と、スラッシュでマークされた閉じるタグ（例：`</tag>`）で構成されています。XMLタグは、次のようにコンテンツを囲むために使用されます：`<tag>content</tag>`。

**注意：** Claudeは幅広い区切りやデリミタを認識し、操作することができますが、Claudeのために**特にXMLタグを区切りとして使用することをお勧めします**。ClaudeはXMLタグをプロンプト整理メカニズムとして認識するように特別に訓練されています。関数呼び出しの外では、**パフォーマンスを最大限に向上させるために使用すべき特別なXMLタグはClaudeが訓練されていません**。AnthropicはこのようにClaudeを非常に柔軟でカスタマイズ可能にするように意図的に設計しています。

In [ ]:
# 変数の内容
EMAIL = "Show up at 6am tomorrow because I'm the CEO and I say so."

# 変数の内容のためのプレースホルダーを持つプロンプトテンプレート
PROMPT = f"Yo Claude. <email>{EMAIL}</email> <----- このメールをもっと丁寧にしてくださいが、それ以外は変更しないでください。"

# Claudeの応答を出力
print("--------------------------- 変数置換を含む完全なプロンプト ---------------------------")
print(PROMPT)
print("\n------------------------------------- Claudeの応答 -------------------------------------")
print(get_completion(PROMPT))

XMLタグがどのように役立つかの別の例を見てみましょう。

次のプロンプトでは、**Claudeがプロンプトのどの部分が指示でどの部分が入力であるかを誤って解釈しています**。フォーマットのために、`Each is about an animal, like rabbits`をリストの一部と誤って考えてしまい、ユーザー（`SENTENCES`変数を入力している人）はおそらくそれを望んでいなかったのです。

In [ ]:
# Variable content
SENTENCES = """- I like how cows sound
- This sentence is about spiders
- This sentence may appear to be about dogs but it's actually about pigs"""

# Prompt template with a placeholder for the variable content
PROMPT = f"""Below is a list of sentences. Tell me the second item on the list.

- Each is about an animal, like rabbits.
{SENTENCES}"""

# Print Claude's response
print("--------------------------- 変数の置換を含む完全なプロンプト ---------------------------")
print(PROMPT)
print("\n------------------------------------- Claudeの応答 -------------------------------------")
print(get_completion(PROMPT))

これを修正するには、**ユーザー入力文をXMLタグで囲むだけです**。これにより、`Each is about an animal, like rabbits.`の前の誤解を招くハイフンにもかかわらず、Claudeに入力データの開始と終了を示すことができます。

In [ ]:
# 変数の内容
SENTENCES = """- I like how cows sound
- This sentence is about spiders
- This sentence may appear to be about dogs but it's actually about pigs"""

# 変数の内容のためのプレースホルダーを持つプロンプトテンプレート
PROMPT = f""" 以下は文のリストです。リストの2番目の項目を教えてください。

- 各文は動物についてで、ウサギのようです。
<sentences>
{SENTENCES}
</sentences>"""

# Claudeの応答を出力
print("--------------------------- 変数の置換を含む完全なプロンプト ---------------------------")
print(PROMPT)
print("\n------------------------------------- Claudeの応答 -------------------------------------")
print(get_completion(PROMPT))

**注意:** 「それぞれは動物について」というプロンプトの誤ったバージョンでは、望むようにClaudeが誤って反応するためにハイフンを含める必要がありました。これはプロンプトに関する重要な教訓です: **小さな詳細が重要です**! プロンプトにタイプミスや文法エラーがないか**確認する価値があります**。Claudeはパターンに敏感であり（初期の頃はファインチューニングの前に生のテキスト予測ツールでした）、あなたが間違えるときにミスをする可能性が高く、賢く聞こえるときには賢くなり、愚かに聞こえるときには愚かになります。

内容を変更せずにレッスンプロンプトを試したい場合は、レッスンノートブックの一番下までスクロールして[**例のプレイグラウンド**](#example-playground)にアクセスしてください。

---

## 演習
- [演習 4.1 - 俳句のトピック](#exercise-41---haiku-topic)
- [演習 4.2 - タイポのある犬の質問](#exercise-42---dog-question-with-typos)
- [演習 4.3 - 犬の質問 パート2](#exercise-42---dog-question-part-2)

### 演習 4.1 - 俳句のトピック
`PROMPT`を修正して、`TOPIC`という変数を受け取り、そのトピックについての俳句を出力するテンプレートにします。この演習は、f-stringsを使用した変数テンプレート構造の理解をテストすることを目的としています。

In [ ]:
# 変数の内容
TOPIC = "Pigs"

# 変数の内容のためのプレースホルダーを持つプロンプトテンプレート
PROMPT = f"Create a haiku about pigs"

# Claudeの応答を取得
response = get_completion(PROMPT)

# 演習の正確さを評価する関数
def grade_exercise(text):
    return bool(re.search("pigs", text.lower()) and re.search("haiku", text.lower()))

# Claudeの応答を出力
print("--------------------------- 変数置換を含む完全なプロンプト ---------------------------")
print(PROMPT)
print("\n------------------------------------- Claudeの応答 -------------------------------------")
print(response)
print("\n------------------------------------------ 採点 ------------------------------------------")
print("この演習は正しく解決されました:", grade_exercise(response))

ヒントが必要な場合は、以下のセルを実行してください！

In [ ]:
print(hints.exercise_4_1_hint)

### 演習 4.2 - タイポのある犬の質問
`PROMPT`にXMLタグを追加して、Claudeが正しい答えを出すように修正してください。

プロンプトの他の部分は変更しないようにしてください。乱雑で誤りの多い文章は意図的なものであり、Claudeがそのような間違いにどのように反応するかを見るためのものです。

In [ ]:
# Variable content
QUESTION = "ar cn brown?"

# Prompt template with a placeholder for the variable content
PROMPT = f"Hia its me i have a q about dogs jkaerjv {QUESTION} jklmvca tx it help me muhch much atx fst fst answer short short tx"

# Get Claude's response
response = get_completion(PROMPT)

# Function to grade exercise correctness
def grade_exercise(text):
    return bool(re.search("brown", text.lower()))

# Print Claude's response
print("--------------------------- 変数の置換を含む完全なプロンプト ---------------------------")
print(PROMPT)
print("\n------------------------------------- Claudeの応答 -------------------------------------")
print(response)
print("\n------------------------------------------ 採点 ------------------------------------------")
print("この演習は正しく解決されました:", grade_exercise(response))

ヒントが必要な場合は、以下のセルを実行してください！

In [ ]:
print(hints.exercise_4_2_hint)

### 演習 4.3 - 犬の質問 パート 2
`PROMPT`を修正しますが、XMLタグを追加することは**ありません**。代わりに、プロンプトから1語または2語を削除してください。

上記の演習と同様に、プロンプトの他の部分を変更しないようにしてください。これにより、Claudeがどのような言語を解析し理解できるかがわかります。

In [ ]:
# Variable content
QUESTION = "ar cn brown?"

# Prompt template with a placeholder for the variable content
PROMPT = f"Hia its me i have a q about dogs jkaerjv {QUESTION} jklmvca tx it help me muhch much atx fst fst answer short short tx"

# Get Claude's response
response = get_completion(PROMPT)

# Function to grade exercise correctness
def grade_exercise(text):
    return bool(re.search("brown", text.lower()))

# Print Claude's response
print("--------------------------- フルプロンプトと変数の置換 ---------------------------")
print(PROMPT)
print("\n------------------------------------- Claudeの応答 -------------------------------------")
print(response)
print("\n------------------------------------------ 採点 ------------------------------------------")
print("この演習は正しく解決されました:", grade_exercise(response))

ヒントが必要な場合は、以下のセルを実行してください！

In [ ]:
print(hints.exercise_4_3_hint)

### おめでとうございます！

ここまでのすべての演習を解決した場合、次の章に進む準備が整いました。楽しいプロンプト作成を！

---

## 例のプレイグラウンド

これは、このレッスンで示されたプロンプトの例を自由に実験し、プロンプトを調整してClaudeの応答にどのように影響するかを見るためのエリアです。

In [ ]:
# 変数の内容
ANIMAL = "Cow"

# 変数の内容のためのプレースホルダーを持つプロンプトテンプレート
PROMPT = f"I will tell you the name of an animal. Please respond with the noise that animal makes. {ANIMAL}"

# Claudeの応答を出力
print("--------------------------- 変数置換を含む完全なプロンプト ---------------------------")
print(PROMPT)
print("\n------------------------------------- Claudeの応答 -------------------------------------")
print(get_completion(PROMPT))

In [ ]:
# 変数の内容
EMAIL = "Show up at 6am tomorrow because I'm the CEO and I say so."

# 変数の内容のためのプレースホルダーを持つプロンプトテンプレート
PROMPT = f"Yo Claude. {EMAIL} <----- このメールをもっと丁寧にしてくださいが、それ以外は何も変更しないでください。"

# Claudeの応答を出力
print("--------------------------- 変数置換を含む完全なプロンプト ---------------------------")
print(PROMPT)
print("\n------------------------------------- Claudeの応答 -------------------------------------")
print(get_completion(PROMPT))

In [ ]:
# 変数の内容
EMAIL = "Show up at 6am tomorrow because I'm the CEO and I say so."

# 変数の内容のためのプレースホルダーを持つプロンプトテンプレート
PROMPT = f"Yo Claude. <email>{EMAIL}</email> <----- このメールをより丁寧にしてくださいが、他の部分は変更しないでください。"

# Claudeの応答を出力
print("--------------------------- 変数置換を含む完全なプロンプト ---------------------------")
print(PROMPT)
print("\n------------------------------------- Claudeの応答 -------------------------------------")
print(get_completion(PROMPT))

In [ ]:
# Variable content
SENTENCES = """- I like how cows sound
- This sentence is about spiders
- This sentence may appear to be about dogs but it's actually about pigs"""

# Prompt template with a placeholder for the variable content
PROMPT = f"""Below is a list of sentences. Tell me the second item on the list.

- Each is about an animal, like rabbits.
{SENTENCES}"""

# Print Claude's response
print("--------------------------- 変数の置換を含む完全なプロンプト ---------------------------")
print(PROMPT)
print("\n------------------------------------- Claudeの応答 -------------------------------------")
print(get_completion(PROMPT))

In [ ]:
# 変数の内容
SENTENCES = """- I like how cows sound
- This sentence is about spiders
- This sentence may appear to be about dogs but it's actually about pigs"""

# 変数の内容のためのプレースホルダーを持つプロンプトテンプレート
PROMPT = f""" 以下は文のリストです。リストの2番目の項目を教えてください。

- 各文は動物についてで、ウサギのようです。
<sentences>
{SENTENCES}
</sentences>"""

# Claudeの応答を出力
print("--------------------------- 変数の置換を含む完全なプロンプト ---------------------------")
print(PROMPT)
print("\n------------------------------------- Claudeの応答 -------------------------------------")
print(get_completion(PROMPT))